<a href="https://colab.research.google.com/github/yihaozhong/Mining-Massive-Data-Sets/blob/main/CS246_Colab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS246 - Colab 1
## Word Count in Spark

### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [2]:
!pip install pyspark
!pip install -U -q PyDrive2
#the output 'xxx is not a symbolic link' will not affect your implementation or execution
#to fix 'xxx is not a symbolic link', you can comment out the lines starting from !mv xxxx
#you may need to replace xxx.11 with the correct version if other errors come up after colab update
#to get the correct version, use !ls /usr/local/lib to find out
!mv /usr/local/lib/libtbbmalloc_proxy.so.2 /usr/local/lib/libtbbmalloc_proxy.so.2.backup
!mv /usr/local/lib/libtbbmalloc.so.2 /usr/local/lib/libtbbmalloc.so.2.backup
!mv /usr/local/lib/libtbbbind_2_5.so.3 /usr/local/lib/libtbbbind_2_5.so.3.backup
!mv /usr/local/lib/libtbb.so.12 /usr/local/lib/libtbb.so.12.backup
!mv /usr/local/lib/libtbbbind_2_0.so.3 /usr/local/lib/libtbbbind_2_0.so.3.backup
!mv /usr/local/lib/libtbbbind.so.3 /usr/local/lib/libtbbbind.so.3.backup
!ln -s /usr/local/lib/libtbbmalloc_proxy.so.2.11 /usr/local/lib/libtbbmalloc_proxy.so.2
!ln -s /usr/local/lib/libtbbmalloc.so.2.11 /usr/local/lib/libtbbmalloc.so.2
!ln -s /usr/local/lib/libtbbbind_2_5.so.3.11 /usr/local/lib/libtbbbind_2_5.so.3
!ln -s /usr/local/lib/libtbb.so.12.11 /usr/local/lib/libtbb.so.12
!ln -s /usr/local/lib/libtbbbind_2_0.so.3.11 /usr/local/lib/libtbbbind_2_0.so.3
!ln -s /usr/local/lib/libtbbbind.so.3.11 /usr/local/lib/libtbbbind.so.3
# !sudo ldconfig
#If error related to the above execution occurs, you can try commenting out the above 12 lines under pip install PyDrive2 (not included)
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64" # changed from java-8 to java-17!
#the output 'xxx is not a symbolic link' will not affect your implementation or execution
#to fix 'xxx is not a symbolic link', you can comment out the lines starting from !mv xxxx
#you may need to replace xxx.11 with the correct version if other errors come up after colab update
#to get the correct version, use !ls /usr/local/lib to find out


openjdk-8-jdk-headless is already the newest version (8u472-ga-1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.


Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [3]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
id='1SE6k_0YukzGd5wK-E4i6mG83nydlfvSa'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('pg100.txt')

If you executed the cells above, you should be able to see the file *pg100.txt* under the "Files" tab on the left panel.

### Your task

If you successfully run the setup stage, you are ready to work on the *pg100.txt* file which contains a copy of the complete works of Shakespeare.

Write a Spark application which outputs the number of words that start with each letter. This means that for every letter, we want to count the total number of (non-unique) words that start with a specific letter. (If a specific (aka unique) word that starts with letter 'a' appears N times, it should be counted in words starting with 'a' N times.)

In your implementation, **ignore the letter case**, i.e., consider all words as lower case. Also, you can ignore all words that **start** with a non-alphabetic character. You should output word counts for the **entire document**, inclusive of the title, author, and the main texts. If you encounter words broken as a result of new lines, e.g. "pro-ject" where the segment after the dash sign is on a new line, no special processing is needed and you can safely consider it as two words ("pro" and "ject").

Your outputs will be graded on a range -- if your differences from the ground-truths are within an error threshold of 5, you'll be considered correct.

**Hint:**
1. split only on space (' ') but not hyphen/dash ('-') or other symbols.
2. you may find spark functions explode (https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.explode.html) and split (https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.split.html) helpful, but you don't need to restrict to them as long as you can satisfy your goal.

Clarification:

1. If a word 'project' is separated into two lines in the form of 'pro-' in the first line and 'ject' in the second line, it should be treated as two words (when you import the text using spark.read.text, it treats each newline as a new row in the DataFrame). However, for the word 'self-love' that appears in a single line, it should be treated as one word starting with letter 's'.

2. As a sanity check, the number of words that start with "b" should be 45455. Note that you will be considered correct as long as your answer is within the error threshold of 5.

In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [6]:
# YOUR
txt = spark.read.text("pg100.txt")

In [7]:
# CODE
txt.take(3)

[Row(value='The Project Gutenberg EBook of The Complete Works of William Shakespeare, by'),
 Row(value='William Shakespeare'),
 Row(value='')]

In [9]:
txt.printSchema()

root
 |-- value: string (nullable = true)



In [17]:
# HERE
txt_splitted = txt.select("*", split(str=col("value"), pattern=" ").alias("words"))

txt_splitted.show()

+--------------------+--------------------+
|               value|               words|
+--------------------+--------------------+
|The Project Guten...|[The, Project, Gu...|
| William Shakespeare|[William, Shakesp...|
|                    |                  []|
|This eBook is for...|[This, eBook, is,...|
|almost no restric...|[almost, no, rest...|
|re-use it under t...|[re-use, it, unde...|
|with this eBook o...|[with, this, eBoo...|
|                    |                  []|
|** This is a COPY...|[**, This, is, a,...|
|**     Please fol...|[**, , , , , Plea...|
|                    |                  []|
|Title: The Comple...|[Title:, The, Com...|
|                    |                  []|
|Author: William S...|[Author:, William...|
|                    |                  []|
|Posting Date: Sep...|[Posting, Date:, ...|
|Release Date: Jan...|[Release, Date:, ...|
|                    |                  []|
|   Language: English|[Language:, English]|
|                    |          

Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!

In [24]:
txt_splitted_exploded = txt_splitted.select("*", explode(col="words").alias("word"))

txt_splitted_exploded.show()

+--------------------+--------------------+------------+
|               value|               words|        word|
+--------------------+--------------------+------------+
|The Project Guten...|[The, Project, Gu...|         The|
|The Project Guten...|[The, Project, Gu...|     Project|
|The Project Guten...|[The, Project, Gu...|   Gutenberg|
|The Project Guten...|[The, Project, Gu...|       EBook|
|The Project Guten...|[The, Project, Gu...|          of|
|The Project Guten...|[The, Project, Gu...|         The|
|The Project Guten...|[The, Project, Gu...|    Complete|
|The Project Guten...|[The, Project, Gu...|       Works|
|The Project Guten...|[The, Project, Gu...|          of|
|The Project Guten...|[The, Project, Gu...|     William|
|The Project Guten...|[The, Project, Gu...|Shakespeare,|
|The Project Guten...|[The, Project, Gu...|          by|
| William Shakespeare|[William, Shakesp...|     William|
| William Shakespeare|[William, Shakesp...| Shakespeare|
|                    |         

In [27]:
word = txt_splitted_exploded.select(lower("word").alias("word_lower"))

word.show()

+------------+
|  word_lower|
+------------+
|         the|
|     project|
|   gutenberg|
|       ebook|
|          of|
|         the|
|    complete|
|       works|
|          of|
|     william|
|shakespeare,|
|          by|
|     william|
| shakespeare|
|            |
|        this|
|       ebook|
|          is|
|         for|
|         the|
+------------+
only showing top 20 rows


In [30]:
letters = word.select(substring("word_lower", 1, 1).alias("letter"))

letters.show()

+------+
|letter|
+------+
|     t|
|     p|
|     g|
|     e|
|     o|
|     t|
|     c|
|     w|
|     o|
|     w|
|     s|
|     b|
|     w|
|     s|
|      |
|     t|
|     e|
|     i|
|     f|
|     t|
+------+
only showing top 20 rows


In [42]:
counting = letters.groupBy("letter").agg(count("*").alias("letter_count")).sort(asc("letter"))

counting.show(100)

+------+------------+
|letter|letter_count|
+------+------------+
|      |      506610|
|     "|         356|
|     #|           3|
|     $|           1|
|     &|          21|
|     '|        3804|
|     (|         639|
|     *|          24|
|     -|          52|
|     .|          52|
|     /|           2|
|     0|           6|
|     1|         458|
|     2|          95|
|     3|          59|
|     4|          46|
|     5|          35|
|     6|          22|
|     7|          17|
|     8|          15|
|     9|          28|
|     :|           1|
|     <|         248|
|     ?|           2|
|     [|        2073|
|     ]|           7|
|     _|           1|
|     a|       84836|
|     b|       45455|
|     c|       34567|
|     d|       29713|
|     e|       18697|
|     f|       36814|
|     g|       20782|
|     h|       60563|
|     i|       62167|
|     j|        3339|
|     k|        9418|
|     l|       29569|
|     m|       55676|
|     n|       26759|
|     o|       43494|
|     p|  